# Analysis of Surface Fields

`mom6_tools.MOM6grid` returns an object with MOM6 grid data.

`mom6_tools.latlon_analysis` has a collection of tools used to perform spatial analysis  (e.g., time averages and spatial mean).

The goal of this notebook is the following:

1) server as an example of how to post-process CESM/MOM6 output;

2) create time averages of surface fields;

3) create time-series of globally-averaged surface fields;

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
import os
import warnings

from dask.distributed import Client, LocalCluster
import intake
from mom6_tools.MOM6grid import MOM6grid

try:
    from mom6_tools.surface import get_MLD, get_BLD

    mom6_import_fail = False
except:
    mom6_import_fail = True
    print(
        "WARNING: Mixed Layer Depth and Boundary Layer Depth calculations will not be computed"
    )
import xarray as xr

warnings.filterwarnings("ignore")

In [ ]:
CESM_output_dir = ""
serial = False  # use dask LocalCluster
case_name = ""
savefigs = False
start_date = ""
end_date = ""
mom6_tools_config = {}
lc_kwargs = {}

In [ ]:
OUTDIR = f"{CESM_output_dir}/{case_name}/ocn/hist/"
print("Output directory is:", OUTDIR)

In [ ]:
# The following parameters must be set accordingly
######################################################

# create an empty class object
class args:
    pass


args.start_date = start_date
args.end_date = end_date
args.casename = case_name
args.native = f"{case_name}.{mom6_tools_config['Fnames']['native']}"
args.static = f"{case_name}.{mom6_tools_config['Fnames']['static']}"
args.mld_obs = "mld-deboyer-tx2_3v2"
args.savefigs = savefigs

In [ ]:
if not os.path.isdir("PNG/BLD"):
    print("Creating a directory to place figures (PNG/BLD)... \n")
    os.system("mkdir -p PNG/BLD")
if not os.path.isdir("PNG/MLD"):
    print("Creating a directory to place figures (PNG/MLD)... \n")
    os.system("mkdir -p PNG/MLD")
if not os.path.isdir("ncfiles"):
    print("Creating a directory to place netcdf files (ncfiles)... \n")
    os.system("mkdir ncfiles")

In [ ]:
# Spin up cluster (if running in parallel)
client = None
if not serial:
    cluster = LocalCluster(**lc_kwargs)
    client = Client(cluster)

client

In [ ]:
# load mom6 grid
grd = MOM6grid(OUTDIR + args.static)
grd_xr = MOM6grid(OUTDIR + args.static, xrformat=True)

In [ ]:
print("Reading native dataset...")
startTime = datetime.now()


def preprocess(ds):
    """Compute montly averages and return the dataset with variables"""
    variables = ["oml", "mlotst", "tos", "SSH", "SSU", "SSV", "speed", "time_bnds"]
    for v in variables:
        if v not in ds.variables:
            ds[v] = xr.zeros_like(ds.SSH)
    return ds[variables]


ds1 = xr.open_mfdataset(OUTDIR + args.native, parallel=False)
ds = preprocess(ds1)

print("Time elasped: ", datetime.now() - startTime)

In [ ]:
print("Selecting data between {} and {}...".format(args.start_date, args.end_date))
ds_sel = ds.sel(time=slice(args.start_date, args.end_date))

In [ ]:
catalog = intake.open_catalog(mom6_tools_config["oce_cat"])
mld_obs = catalog[args.mld_obs].to_dask()
# uncomment to list all datasets available
# list(catalog)

### Mixed layer depth

In [ ]:
%matplotlib inline
# MLD
if not mom6_import_fail:
    get_MLD(ds, "mlotst", mld_obs, grd, args)

### Boundary layer depth

In [ ]:
if not mom6_import_fail:
    get_BLD(ds, "oml", grd, args)

In [ ]:
# SSH (not working)
# get_SSH(ds, 'SSH', grd, args)